# Query DAS after loading a knowledge base

This notebook shows how to instantiate a DAS with some knowledge base already loaded and query it.

The first cell just imports the relevant class and instantiates relevant objects.

In [ ]:
from das.distributed_atom_space import DistributedAtomSpace, QueryOutputFormat
from das.pattern_matcher.pattern_matcher import PatternMatchingAnswer, OrderedAssignment, UnorderedAssignment, CompositeAssignment, Node, Link, Variable, Not, And, Or
import warnings
warnings.filterwarnings('ignore')
das = DistributedAtomSpace()
db = das.db
db.prefetch()

Follows two utilitary functions which are an example of how to iterate and manipulate a query answer.

Query answers may have two types of assignments. `Ordered` and `unordered`. The former is a regular assignment defining a value for each variable in the query. For instance:

```
V1 = H1
V2 = H2
V3 = H3
```

The later assigns a set of possible values to a set of variables. For instance:

```
V1, V2, V3 = H1, H2, H3
```

It means that any of these assignments are a valid answer to the query:

```
V1 = H1
V2 = H2
V3 = H3

V1 = H1
V2 = H3
V3 = H2

V1 = H2
V2 = H1
V3 = H3

V1 = H2
V2 = H3
V3 = H1

V1 = H3
V2 = H1
V3 = H2

V1 = H3
V2 = H2
V3 = H1
```

In [ ]:
def print_ordered_assignment(assignment):
    if assignment is not None:
        for key, value in assignment.mapping.items():
            print(f"{key}: {db.get_node_name(value)}")

def print_unordered_assignment(assignment):
    if assignment is not None:
        symbols = []
        for key in assignment.symbols:
            for i in range(assignment.symbols[key]):
                symbols.append(key)
        values = []
        for key in assignment.values:
            for i in range(assignment.values[key]):
                values.append(key)
        mapping_keys = []
        mapping_values = []
        for symbol, value in zip(symbols, values):
            mapping_keys.append(symbol)
            mapping_values.append(db.get_node_name(value))
        print(f"{mapping_keys} = {mapping_values}")

Before proceeding, print the atom count to make sure the knowledge base is correct.

In [ ]:
das.count_atoms()

Now we define 4 example queries to use later. We are just building the queries here, not executing them.

Each query is an object built using its constructor and possibly nesting other constructors in its arguments. We can use logical operators (`And`, `Or` and `Not`), nodes (`Node`), links (`Link`) and variables (`Variable` or `TypedVariable`)

As constructor's parameters, `And` and `Or` expect a list of other logical operators or `Links`. `Not` expects a single logical operator or a `Link`.

`Links` expect a type, a flag to indicated if targets ar ordered or not and a list of targets.

The list of targets of a link may contain `Nodes` or `Variables`.

In `my_query_1` we define a query which is a simple `AND` operation on two links whose targets are variables. We are looking for two inheritance links which links 3 nodes in such a way that V1 -> V2 -> V3.

In [ ]:
V1 = Variable("V1")
V2 = Variable("V2")
V3 = Variable("V3")
my_query_1 = And([
    Link("Inheritance", ordered=True, targets=[V1, V2]),
    Link("Inheritance", ordered=True, targets=[V2, V3])
])

In `my_quary_2` we are doing a similar query but adding an extra constraint that it shaw not exist an inheritance link "human" -> V2

In [ ]:
V1 = Variable("V1")
V2 = Variable("V2")
V3 = Variable("V3")
N1 = Node("Concept", "human")
my_query_2 = And([
    Link("Inheritance", ordered=True, targets=[V1, V2]),
    Link("Inheritance", ordered=True, targets=[V2, V3]),
    Not(Link("Inheritance", ordered=True, targets=[N1, V2]))
])

In `my_query_3` we do a similar query but now there shaw not exist neither an inheritance link "human" -> V2 nor "snake" -> V2.

In [ ]:
V1 = Variable("V1")
V2 = Variable("V2")
V3 = Variable("V3")
N1 = Node("Concept", "human")
N2 = Node("Concept", "snake")
my_query_3 = And([
    Link("Inheritance", ordered=True, targets=[V1, V2]),
    Link("Inheritance", ordered=True, targets=[V2, V3]),
    Not(Or([
        Link("Inheritance", ordered=True, targets=[N1, V2]),
        Link("Inheritance", ordered=True, targets=[N2, V2])
    ]))
])

In `my_query_4` we show an example of query using an unordered link. We are looking for nodes V1 and V2 which have a similarity link between them but which don't have inheritance links fromk any of them to "mammal".

In [ ]:
V1 = Variable("V1")
V2 = Variable("V2")
V3 = Variable("V3")
N1 = Node("Concept", "mammal")
my_query_4 = And([
    Link("Similarity", ordered=False, targets=[V1, V2]),
    Not(Or([
        Link("Inheritance", ordered=True, targets=[V1, N1]),
        Link("Inheritance", ordered=True, targets=[V2, N1]),
    ]))
])

Now select which query you want to execute and see the results. You can re-execute the cells below just changing the query (no need to re-execute the previous cells).

In [ ]:
my_query = my_query_1

In [ ]:
query_answer = PatternMatchingAnswer()
matched = my_query.matched(db, query_answer)
print(matched)

In [ ]:
query_answer.assignments

In [ ]:
for assignment in query_answer.assignments:
    if type(assignment) is OrderedAssignment:
        print_ordered_assignment(assignment)
    elif type(assignment) is UnorderedAssignment:
        print_unordered_assignment(assignment)
    elif type(assignment) is CompositeAssignment:
        print_ordered_assignment(assignment.ordered_mapping)
        for unordered_assignment in assignment.unordered_mappings:
            print_unordered_assignment(unordered_assignment)
    print("")